# TFSolve - time and frequency domain ODE solvers

This notebook demonstrates solving time-domain equations of motion.

Notes:

* Some features depend on the equations being in modal space (particularly important where there are distinctions between the rigid-body modes and the elastic modes).
* The time-domain solvers all depend on constant time step.

First, do some imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyyeti.tfsolve import TFSolve

Some settings specifically for the jupyter notebook. This and other notebooks are available here: https://github.com/twmacro/pyyeti/tree/master/docs/tutorials.

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [8.0, 6.0]
plt.rcParams['savefig.dpi'] = 100.

#### Setup a simple system
This system is not fixed to ground so it does have a rigid-body mode. This system is from the `frclim.ntfl` example.

Define the mass, damping and stiffness matrices:

In [ ]:
M1 = 10.
M2 = 30.
M3 = 3.
M4 = 2.
c1 = 15.
c2 = 15.
c3 = 15.
k1 = 45000.
k2 = 25000.
k3 = 10000.

MASS = np.array([[M1, 0, 0, 0],
                 [0, M2, 0, 0],
                 [0, 0, M3, 0],
                 [0, 0, 0, M4]])
DAMP = np.array([[c1, -c1, 0, 0],
                 [-c1, c1+c2, -c2, 0],
                 [0, -c2, c2+c3, -c3],
                 [0, 0, -c3, c3]])
STIF = np.array([[k1, -k1, 0, 0],
                 [-k1, k1+k2, -k2, 0],
                 [0, -k2, k2+k3, -k3],
                 [0, 0, -k3, k3]])

Define a time vector and a forcing function:

In [ ]:
h = .001
t = np.arange(0, 2, h)
F = np.zeros((4, len(t)))
F[0, :200] = np.arange(200)
F[0, 200:400] = np.arange(200)[::-1]
F[0, 400:600] = np.arange(0, -200, -1)
F[0, 600:800] = np.arange(0, -200, -1)[::-1]
plt.plot(t, F[0])
plt.xlabel('Time (s)')
plt.ylabel('Force')
plt.grid()

TFSolve was originally designed only to solve systems that were in modal space via the 'su' solver. The 'su' name stands for "solve uncoupled". This is an exact solver assuming piece-wise linear forces. It can now do that conversion to modal space for you by using 'eigsu' instead of 'su' (the underlying ODE solver is the same), or you can choose the 'se2' solver which is more general but typically not as fast. Note, even for the 'se2' solver, the equations will need to be put in modal space (via 'eigse2') if you wish to use static initial conditions.

#### The "su" solver

The recommended way to use TFSolve is to initialize your solver of choice during instantiation, as follows. Note that since this system is not in modal space, we'll use the 'su' solver via 'eigsu'.

In [ ]:
ts = TFSolve('eigsu', MASS, DAMP, STIF, h)

At this point, many pre-calculations are done and the `ts` object is ready to be called (repeatedly if necessary) to solve the equations of motion with arbitrary forces and initial conditions. The time domain solver is called via the method `tsolve`:

In [ ]:
sol = ts.tsolve(F)

`sol` is a SimpleNamespace with the members:

* .a - acceleration
* .v - velocity
* .d - displacement
* .t - time vector
* .h - time step

Plot the acceleration responses (assume metric units):

In [ ]:
for j in range(sol.a.shape[0]):
    plt.plot(sol.t, sol.a[j], label='Acce {:d}'.format(j+1))
plt.legend(loc='best')
plt.xlabel('Time (s)')
plt.ylabel(r'Acceleration (m/$sec^2$)')
plt.grid()

Plot the velocities:

In [ ]:
for j in range(sol.v.shape[0]):
    plt.plot(sol.t, sol.v[j], label='Velocity {:d}'.format(j+1))
plt.legend(loc='best')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/sec)')
plt.grid()

Plot the displacements:

In [ ]:
for j in range(sol.d.shape[0]):
    plt.plot(sol.t, sol.d[j], label='Displacement {:d}'.format(j+1))
plt.legend(loc='best')
plt.xlabel('Time (s)')
plt.ylabel('Displacement (m)')
plt.grid()

---
#### The 'se2' solver

For demonstration, we'll solve the same system using the 'se2' solver. Since we're not using static initial conditions, we can use either 'eigse2' or just 'se2'. In this case, the 'se2' solver would be preferred because it does less work.

First, do the pre-calcs for both:

In [ ]:
ts1 = TFSolve('se2', MASS, DAMP, STIF, h)
ts2 = TFSolve('eigse2', MASS, DAMP, STIF, h)


Solve with both solvers and demonstrate they give the same results:

In [ ]:
sol1 = ts1.tsolve(F)
sol2 = ts2.tsolve(F)

assert np.allclose(sol1.a, sol2.a)
assert np.allclose(sol1.v, sol2.v)
assert np.allclose(sol1.d, sol2.d)

Show the results are also the same as the 'su' solver:

In [ ]:
assert np.allclose(sol1.a, sol.a)
assert np.allclose(sol1.v, sol.v)
assert np.allclose(sol1.d, sol.d)